In [37]:
import pickle
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn import cluster
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

# NIST

In [133]:
from sklearn.datasets import load_digits
mnist = load_digits()
print(mnist.data.shape)

(1797, 64)


In [134]:
X = mnist.data 
y = mnist.target
# X = X.reshape(X.shape[0],8,8)

In [135]:
ndim = 20
nsamp = 3
seeds = [123,1234,12345]

## Test

In [77]:
def testacc(Xtr,ytr,Xt,yt,sel,sample=True):
    knn = KNeighborsClassifier(n_neighbors=1)
    if sample:
        idx = np.random.choice(Xtr.shape[0], 100, replace=False, )
        knn.fit(Xtr[idx][:,sel], ytr[idx])
    else:
        knn.fit(Xtr[:,sel], ytr)
    preds = knn.predict(Xt[:,sel])
    print(sum(preds==yt) / len(yt))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [97]:
# fea sel 1
sel = VarianceThreshold()
sel.fit(X_train)
dimsel = np.argsort(sel.variances_)[-ndim:]
print(dimsel)
testacc(X_train, y_train, X_test, y_test, dimsel)

[45 43 21 20 26 27 28 29 62  3  5 35 36 37  2 19 18 42 44 34]
0.8822222222222222


In [96]:
# fea sel 1
sel = VarianceThreshold()
sel.fit(X_train)
dimsel = []
for d in reversed(np.argsort(sel.variances_)):
    issim = False
    for d2 in dimsel:
        dist = np.sum(np.abs(X_train[:,d] - X_train[:,d2]) > 0.5) / len(X_train)
        if dist < 0.1:
            issim = True
            break
    if issim: continue
    dimsel.append(d)
    if len(dimsel) == ndim:
        break
print(dimsel)
testacc(X_train, y_train, X_test, y_test, dimsel)

[34, 44, 42, 18, 19, 2, 37, 36, 35, 5, 3, 62, 29, 28, 27, 26, 20, 21, 43, 45]
0.8822222222222222


In [113]:
# fea sel 2
agglo = cluster.FeatureAgglomeration(n_clusters=ndim)
agglo.fit(X_train)
dimsel = []
for i in range(ndim):
    for idx,j in enumerate(agglo.labels_):
        if i==j:
            dimsel.append(idx)
            break
testacc(X_train, y_train, X_test, y_test, dimsel)

0.8266666666666667


In [112]:
# fea sel 2
sel = VarianceThreshold()
sel.fit(X_train)
dimsel_ = np.argsort(sel.variances_)[-ndim*2:]
# dimsel_ = np.nonzero(sel.variances_)[0]

agglo = cluster.FeatureAgglomeration(n_clusters=ndim)
agglo.fit(X_train[:,dimsel_])
dimsel = []
for i in range(ndim):
    for idx,j in enumerate(agglo.labels_):
        if i==j:
            dimsel.append(dimsel_[idx])
            break
testacc(X_train, y_train, X_test, y_test, dimsel)

0.8333333333333334


In [132]:
# fea sel 3
model = PCA(n_components=ndim).fit(X_train)
n_pcs = model.components_.shape[0]
dimsel = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]
testacc(X_train, y_train, X_test, y_test, dimsel)

0.8355555555555556


In [130]:
# fea sel 3
model = PCA(n_components=ndim//2).fit(X_train)
n_pcs = model.components_.shape[0]
dimsel = [np.argsort(np.abs(model.components_[i]))[-2:] for i in range(n_pcs)]
dimsel = [i for i in itertools.chain(*dimsel)]
print(dimsel)
testacc(X_train, y_train, X_test, y_test, dimsel)

[42, 34, 44, 28, 21, 29, 61, 10, 21, 42, 19, 52, 35, 27, 61, 13, 36, 45, 18, 36]
0.8377777777777777


## Sel

In [136]:
samps = []
for i,seed in zip(range(nsamp),seeds):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    dims = []
    
    # fea sel 1
    sel = VarianceThreshold()
    sel.fit(X_train)
    dimsel = np.argsort(sel.variances_)[-ndim:]
    dims.append(dimsel)
    
    # fea sel 2
    agglo = cluster.FeatureAgglomeration(n_clusters=ndim)
    agglo.fit(X_train)
    dimsel = []
    for i in range(ndim):
        for idx,j in enumerate(agglo.labels_):
            if i==j:
                dimsel.append(idx)
                break
    dims.append(np.array(dimsel))

    # fea sel 3
    # PCA: https://stackoverflow.com/questions/50796024/feature-variable-importance-after-a-pca-analysis
    model = PCA(n_components=ndim).fit(X_train)
    n_pcs= model.components_.shape[0]
    # get the index of the most important feature on EACH component
    dimsel = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]
    dims.append(np.array(dimsel))

    samps.append([(X_train, y_train), (X_test, y_test), dims])

In [137]:
with open('datasets/nist.pkl', 'wb') as fout:
    pickle.dump(samps, fout)